In [5]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

url = 'https://cafef.vn'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

# Tìm thẻ div chứa menu chính
menu_div = soup.find('div', class_='menucategory menuheader header__nav', id='menu_wrap')
menu_items = menu_div.find_all('li', class_='acvmenu') if menu_div else []

categories = {}

for li in menu_items:
    a_tag = li.find('a')
    if a_tag and 'href' in a_tag.attrs:
        name = a_tag.get_text(strip=True)
        link = a_tag['href']
        if link.startswith('/'):  # nối với domain
            link = 'https://cafef.vn' + link
        if name:  # bỏ qua nếu tên rỗng (như icon trang chủ)
            categories[name] = link

# In ra các chuyên mục
print(f"📚 Tìm thấy {len(categories)} chuyên mục:")
for name, link in categories.items():
    print(f"- {name}: {link}")



📚 Tìm thấy 12 chuyên mục:
- XÃ HỘI: https://cafef.vn/xa-hoi.chn
- CHỨNG KHOÁN: https://cafef.vn/thi-truong-chung-khoan.chn
- BẤT ĐỘNG SẢN: https://cafef.vn/bat-dong-san.chn
- DOANH NGHIỆP: https://cafef.vn/doanh-nghiep.chn
- NGÂN HÀNG: https://cafef.vn/tai-chinh-ngan-hang.chn
- SMART MONEY: https://cafef.vn/smart-money.chn
- TÀI CHÍNH QUỐC TẾ: https://cafef.vn/tai-chinh-quoc-te.chn
- VĨ MÔ: https://cafef.vn/vi-mo-dau-tu.chn
- KINH TẾ SỐ: https://cafef.vn/kinh-te-so.chn
- THỊ TRƯỜNG: https://cafef.vn/thi-truong.chn
- SỐNG: https://cafef.vn/song.chn
- LIFESTYLE: https://cafef.vn/lifestyle.chn


In [6]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Cấu hình Selenium Chrome headless
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Khai báo danh sách chuyên mục cần crawl

num_clicks = 15  # Số lần bấm "Xem thêm"
article_counter = 1
data = []

for category, base_url in categories.items():
    print(f"\n=== 📂 Crawling chuyên mục: {category.upper()} ===")

    driver.get(base_url)
    wait = WebDriverWait(driver, 10)

    for i in range(num_clicks):
        try:
            button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "btn-viewmore")))
            driver.execute_script("arguments[0].click();", button)
            print(f"🔄 Đã bấm 'Xem thêm' lần {i + 1}")
            time.sleep(2)
        except:
            print("⚠️ Không thể bấm nút 'Xem thêm' nữa.")
            break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('div', class_='tlitem box-category-item')

    print(f"📰 Tìm thấy {len(articles)} bài viết:")
    for article in articles:
        h3_tag = article.find('h3')
        a_tag = h3_tag.find('a') if h3_tag else None
        if a_tag and 'href' in a_tag.attrs:
            title = a_tag.get_text(strip=True)
            link = a_tag['href']
            if link.startswith('/'):
                link = 'https://cafef.vn' + link
            print(f"- {title}: {link}")

            try:
                article_res = requests.get(link)
                article_soup = BeautifulSoup(article_res.text, 'html.parser')

                content_div = article_soup.find('div', class_='contentdetail')
                if content_div:
                    paragraphs = content_div.find_all('p')
                    content_text = '\n'.join([p.get_text(strip=True) for p in paragraphs])
                else:
                    content_text = None

                time_tag = article_soup.find('span', class_='pdate')
                date = time_tag.get_text(strip=True) if time_tag else None

                author = None
                normal_paragraphs = article_soup.find_all('p', class_='Normal')
                for p in reversed(normal_paragraphs):
                    strong_tag = p.find('strong')
                    if strong_tag:
                        possible_author = strong_tag.get_text(strip=True)
                        if possible_author and len(possible_author.split()) <= 5:
                            author = possible_author
                            break

                if not author:
                    author_tag = article_soup.find('p', class_='author')
                    if author_tag:
                        author = author_tag.get_text(strip=True)

                article_id = f"CAFEF_{article_counter:02d}"
                article_counter += 1

                data.append({
                    'id': article_id,
                    'title': title,
                    'link': link,
                    'content': content_text,
                    'date': date,
                    'author': author,
                    'category': category,
                    'source': 'CafeF'
                })

            except Exception as e:
                print(f"❌ Lỗi bài viết: {e}")

            time.sleep(1)

driver.quit()

df = pd.DataFrame(data)



=== 📂 Crawling chuyên mục: XÃ HỘI ===
🔄 Đã bấm 'Xem thêm' lần 1
🔄 Đã bấm 'Xem thêm' lần 2
🔄 Đã bấm 'Xem thêm' lần 3
🔄 Đã bấm 'Xem thêm' lần 4
🔄 Đã bấm 'Xem thêm' lần 5
🔄 Đã bấm 'Xem thêm' lần 6
🔄 Đã bấm 'Xem thêm' lần 7
🔄 Đã bấm 'Xem thêm' lần 8
🔄 Đã bấm 'Xem thêm' lần 9
🔄 Đã bấm 'Xem thêm' lần 10
🔄 Đã bấm 'Xem thêm' lần 11
🔄 Đã bấm 'Xem thêm' lần 12
🔄 Đã bấm 'Xem thêm' lần 13
🔄 Đã bấm 'Xem thêm' lần 14
🔄 Đã bấm 'Xem thêm' lần 15
📰 Tìm thấy 238 bài viết:
- Các chủ xe phạm lỗi chạy quá tốc độ nhanh chóng nộp phạt nguội theo Nghị định 168: https://cafef.vn/cac-chu-xe-pham-loi-chay-qua-toc-do-nhanh-chong-nop-phat-nguoi-theo-nghi-dinh-168-188250417140802058.chn
- Bác sĩ nghi xâm hại tình dục bệnh nhân ở Hà Nam có thể đối mặt mức án nào?: https://cafef.vn/bac-si-nghi-xam-hai-tinh-duc-benh-nhan-o-ha-nam-co-the-doi-mat-muc-an-nao-188250417140049891.chn
- Bắt một giám đốc mỏ đất: https://cafef.vn/bat-mot-giam-doc-mo-dat-188250417141257421.chn
- Chính phủ đề xuất nâng mức hỗ trợ tiền ăn trưa c

In [7]:
df

,id,title,link,content,date,author,category,source
0,CAFEF_01,Các chủ xe phạm lỗi chạy quá tốc độ nhanh chón...,https://cafef.vn/cac-chu-xe-pham-loi-chay-qua-...,"Từ ngày 1/3/2025 đến ngày 31/3/2025, thông qua...",17-04-2025 - 14:30 PM,Theo Trang Anh,XÃ HỘI,CafeF
1,CAFEF_02,Bác sĩ nghi xâm hại tình dục bệnh nhân ở Hà Na...,https://cafef.vn/bac-si-nghi-xam-hai-tinh-duc-...,"Ngày 14/4, Cơ quan Cảnh sát điều tra Công an t...",17-04-2025 - 14:15 PM,Theo Chi Chi,XÃ HỘI,CafeF
2,CAFEF_03,Bắt một giám đốc mỏ đất,https://cafef.vn/bat-mot-giam-doc-mo-dat-18825...,Văn phòng Cơ quan Cảnh sát điều tra Công an tỉ...,17-04-2025 - 14:12 PM,Theo Tuấn Minh,XÃ HỘI,CafeF
3,CAFEF_04,Chính phủ đề xuất nâng mức hỗ trợ tiền ăn trưa...,https://cafef.vn/chinh-phu-de-xuat-nang-muc-ho...,"Sáng 17-4, tiếp tục chương trình Phiên họp thứ...",17-04-2025 - 13:56 PM,Theo Văn Duẩn,XÃ HỘI,CafeF
4,CAFEF_05,Cách hết tất cả chức vụ trong Đảng đối với bà ...,https://cafef.vn/cach-het-tat-ca-chuc-vu-trong...,Bà Nguyễn Thị Phương Lan - Chi cục trưởng Chi ...,17-04-2025 - 13:34 PM,Theo Hoàng Phúc,XÃ HỘI,CafeF
...,...,...,...,...,...,...,...,...
2690,CAFEF_2691,Tôi đã bán nhà ở TP.HCM để về sống ở quê – và ...,https://cafef.vn/toi-da-ban-nha-o-tphcm-de-ve-...,"Ở tuổi 46, khi phần lớn bạn bè đang gồng mình ...",13-04-2025 - 10:29 AM,Theo Như Anh,LIFESTYLE,CafeF
2691,CAFEF_2692,5 khoảnh khắc đưa Ánh Viên thành mỹ nhân có 2 ...,https://cafef.vn/5-khoanh-khac-dua-anh-vien-th...,"Không phải là gương mặt xa lạ, Ánh Viên (SN 19...",13-04-2025 - 10:22 AM,Theo Hải My,LIFESTYLE,CafeF
2692,CAFEF_2693,"Cuộc sống của nữ MC thể thao xinh đẹp ""đi nhầm...",https://cafef.vn/cuoc-song-cua-nu-mc-the-thao-...,\nMC Thu Hoài là gương mặt quen thuộc với khán...,13-04-2025 - 10:01 AM,Theo Hương Giang,LIFESTYLE,CafeF
2693,CAFEF_2694,"Trùm Điền Quân Color Man đi xin việc, bị lãnh ...",https://cafef.vn/trum-dien-quan-color-man-di-x...,"Mới đây, ông trùm đế chế truyền thông giải trí...",13-04-2025 - 09:51 AM,Theo Tùng Ninh,LIFESTYLE,CafeF


In [8]:
df.to_csv('cafef_articles.csv', index=False, encoding='utf-8-sig')
print("Data exported to 'cafef_articles.csv'")

Data exported to 'cafef_articles.csv'
